In [3]:
import numpy as np
import urllib
import os
import logging
from pathlib import Path
import pandas as pd
import regex as re
from sklearn.linear_model import LinearRegression

In [11]:
re.findall('[Aa]r','are')

['ar']

In [12]:
# Load in csvs
WHR_file_name = 'world-happiness-report.csv'
WHR2021_file_name = 'world-happiness-report-2021.csv'
CPDS_file_name = 'cleaned_CPDS.xlsx'
wealth_file_name = 'wealth_data.csv'
WHR_df = pd.read_csv(f'data\{WHR_file_name}')
WHR2021_df = pd.read_csv(f'data\{WHR2021_file_name}')
CPDS_df = pd.read_excel(f'data\{CPDS_file_name}')
wealth_df = pd.read_csv(f'data\{wealth_file_name}')


# Clean Data
WHR_df = WHR_df.drop(columns = ['Positive affect','Negative affect'])
WHR2021_df['year'] = 2021
WHR2021_df = WHR2021_df.drop(columns = ['Standard error of ladder score','upperwhisker','lowerwhisker','Ladder score in Dystopia',
                          'Explained by: Log GDP per capita','Explained by: Social support','Explained by: Healthy life expectancy',
                          'Explained by: Freedom to make life choices','Explained by: Generosity','Explained by: Perceptions of corruption',
                          'Dystopia + residual'])
WHR2021_df = WHR2021_df.rename(columns = {'Ladder score':'Life Ladder','Logged GDP per capita':'Log GDP per capita',
                            'Healthy life expectancy':'Healthy life expectancy at birth'})

dic = {}
for i in WHR2021_df['Regional indicator'].unique():
    dic[i]=list(WHR2021_df[WHR2021_df['Regional indicator'] ==i].groupby('Country name').size().index)
dic['Sub-Saharan Africa'].append('Angola')
dic['Latin America and Caribbean'].append('Belize')
dic['South Asia'].append('Bhutan')
dic['Sub-Saharan Africa'].append('Central African Republic')
dic['Sub-Saharan Africa'].append('Congo (Kinshasa)')
dic['Latin America and Caribbean'].append('Cuba')
dic['Sub-Saharan Africa'].append('Djibouti')
dic['Latin America and Caribbean'].append('Guyana')
dic['Middle East and North Africa'].append('Oman')
dic['Middle East and North Africa'].append('Qatar')
dic['Sub-Saharan Africa'].append('Somalia')
dic['Sub-Saharan Africa'].append('Somaliland region')
dic['Sub-Saharan Africa'].append('South Sudan')
dic['Middle East and North Africa'].append('Sudan')
dic['Latin America and Caribbean'].append('Suriname')
dic['Middle East and North Africa'].append('Syria')
dic['Latin America and Caribbean'].append('Trinidad and Tobago')

def find_region(x):
    '''Helper Function to Return Region Name based on the Country Name entered'''
    for reg in dic.keys():
        for c in dic[reg]:
            if x == c:
                return reg
WHR_df['Regional indicator'] = WHR_df['Country name'].apply(find_region)


cmbd_WHR_df = pd.concat([WHR_df,WHR2021_df])


CPDS_df = CPDS_df.dropna(subset = ['year','country'])
def convert_to_int(x):
    return int(x)
def convert_to_float(x):
    if x != x:
        return x
    try:
        return float(x)
    except:
        return x
CPDS_df['year'] = CPDS_df[['year']].applymap(convert_to_int)

def fix_name(x):
    '''fix country names to be in same format as WHR'''
    if x == 'USA':
        return 'United States'
    else:
        return x
CPDS_df['country'] = CPDS_df[['country']].applymap(fix_name)

all_data = pd.merge(cmbd_WHR_df,CPDS_df,how = 'left',left_on = ['year','Country name'],right_on = ['year','country'])
all_data = all_data.drop(columns = ['country'])
#all_data = pd.merge(cmbd_WHR_df,wealth_df, how = 'left',left_on = 'Country name', right_on = 'country')
#all_data = all_data.drop(columns = ['country'])
all_data = all_data.applymap(convert_to_float)
all_data

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Regional indicator,...,gov_type,vturn,womenpar,rae_ele,rae_leg,effpar_ele,effpar_leg,dis_abso,dis_rel,dis_gall
0,Afghanistan,2008.0,3.724,7.370,0.451,50.800,0.718,0.168,0.882,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2009.0,4.402,7.540,0.552,51.200,0.679,0.190,0.850,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2010.0,4.758,7.647,0.539,51.600,0.600,0.121,0.707,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2011.0,3.832,7.620,0.521,51.920,0.496,0.162,0.731,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2012.0,3.783,7.705,0.521,52.240,0.531,0.236,0.776,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,Lesotho,2021.0,3.512,7.926,0.787,48.700,0.715,-0.131,0.915,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,Botswana,2021.0,3.467,9.782,0.784,59.269,0.824,-0.246,0.801,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,Rwanda,2021.0,3.415,7.676,0.552,61.400,0.897,0.061,0.167,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,Zimbabwe,2021.0,3.145,7.943,0.750,56.201,0.677,-0.047,0.821,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:

list(all_data.columns[all_data.dtypes != float].drop(['Country name']))

['Regional indicator']

In [46]:
all_data

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Regional indicator,...,gov_type,vturn,womenpar,rae_ele,rae_leg,effpar_ele,effpar_leg,dis_abso,dis_rel,dis_gall
0,Afghanistan,2008.0,3.724,7.370,0.451,50.800,0.718,0.168,0.882,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2009.0,4.402,7.540,0.552,51.200,0.679,0.190,0.850,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2010.0,4.758,7.647,0.539,51.600,0.600,0.121,0.707,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2011.0,3.832,7.620,0.521,51.920,0.496,0.162,0.731,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2012.0,3.783,7.705,0.521,52.240,0.531,0.236,0.776,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,Lesotho,2021.0,3.512,7.926,0.787,48.700,0.715,-0.131,0.915,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,Botswana,2021.0,3.467,9.782,0.784,59.269,0.824,-0.246,0.801,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,Rwanda,2021.0,3.415,7.676,0.552,61.400,0.897,0.061,0.167,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,Zimbabwe,2021.0,3.145,7.943,0.750,56.201,0.677,-0.047,0.821,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
cmbd_WHR_df[(cmbd_WHR_df['year'] >= 2006) & (cmbd_WHR_df['year'] <= 2021)].isnull().sum()

Country name                          0
year                                  0
Life Ladder                           0
Log GDP per capita                   36
Social support                       13
Healthy life expectancy at birth     55
Freedom to make life choices         30
Generosity                           63
Perceptions of corruption           108
Regional indicator                    0
dtype: int64

In [49]:
wealth_df.sort_values(by='country')['country'].unique()

array(['Albania', 'Algeria', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bangladesh', 'Belgium', 'Bolivia',
       'Brazil', 'Bulgaria', 'Burundi', 'Cambodia', 'Cameroon', 'Canada',
       'Chad', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia',
       'Czech Republic', 'Denmark', 'Dominican Republic', 'Egypt',
       'El Salvador', 'Estonia', 'Finland', 'France', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Guatemala', 'Honduras', 'Hungary',
       'Iceland', 'India', 'Indonesia', 'Iran', 'Ireland', 'Israel',
       'Italy', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kyrgyzstan',
       'Laos', 'Latvia', 'Lesotho', 'Lithuania', 'Luxembourg',
       'Madagascar', 'Malawi', 'Malaysia', 'Mali', 'Mauritania', 'Mexico',
       'Moldova', 'Mongolia', 'Morocco', 'Mozambique', 'Namibia', 'Nepal',
       'Netherlands', 'New Zealand', 'Nicaragua', 'Nigeria', 'Norway',
       'Pakistan', 'Panama', 'Paraguay', 'Peru', 'Philippines', 'Poland',
       'Portugal',

In [37]:
all_data['Country name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Estonia', 'Ethiopia', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Honduras',
       'Hong Kong S.A.R. of China', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Ivory Coast', 'Jamaica', 'Japan', 'Jord

In [25]:
all_data.columns

Index(['Country name', 'year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Regional indicator', 'poco', 'eu', 'emu',
       'gov_right1', 'gov_cent1', 'gov_left1', 'gov_new', 'gov_gap',
       'gov_chan', 'gov_right2', 'gov_cent2', 'gov_left2', 'gov_right3',
       'gov_cent3', 'gov_left3', 'gov_sup', 'gov_type', 'vturn', 'womenpar',
       'rae_ele', 'rae_leg', 'effpar_ele', 'effpar_leg', 'dis_abso', 'dis_rel',
       'dis_gall'],
      dtype='object')

In [40]:
years_present_df = cmbd_WHR_df[(cmbd_WHR_df['year'] >= 2006) & (cmbd_WHR_df['year'] <= 2021)].groupby('Country name').count()[['year']]
countries_to_include = list(years_present_df[years_present_df['year'] > 8].index)

In [6]:
temp_df = cmbd_WHR_df.drop(columns = ['Regional indicator'])

In [57]:

years_present_df = cmbd_WHR_df[(cmbd_WHR_df['year'] >= 2006) & (cmbd_WHR_df['year'] <= 2021)].groupby('Country name').count()[['year']]
countries_to_include = list(years_present_df[years_present_df['year'] > 10].index)
included_years = [i for i in range(2007,2022)]
columns = temp_df.columns[2:]
new_col_names = []
for i in included_years:
    year = str(i)
    for col in columns:
        col_name = f'{col}_{year}'
        new_col_names.append(col_name)

groups = temp_df.groupby('Country name')

num_cols = len(columns)
missing_year_data = [np.nan for i in range(num_cols)]
data = {}
for key, group in groups:
    if key in countries_to_include:
        row_data = []
        group_years = list(group['year'])
        c_group = group.drop(columns = 'Country name')
        c_group = c_group.set_index('year')
        for i in included_years:
            if i in group_years:
                row_data += list(c_group.loc[i])
            else:
                row_data += missing_year_data
                
        data[key] = row_data
    

reformatted_data = pd.DataFrame.from_dict(data, orient='index',columns = new_col_names)

ttl_nulls = pd.DataFrame(reformatted_data.isnull().sum()).sum()
ttl_vals = reformatted_data.shape[0] * reformatted_data.shape[1]
pct_missing = ttl_nulls / ttl_vals
print(pct_missing)

reformatted_data

0    0.095688
dtype: float64


,Life Ladder_2007,Log GDP per capita_2007,Social support_2007,Healthy life expectancy at birth_2007,Freedom to make life choices_2007,Generosity_2007,Perceptions of corruption_2007,Life Ladder_2008,Log GDP per capita_2008,Social support_2008,...,Freedom to make life choices_2020,Generosity_2020,Perceptions of corruption_2020,Life Ladder_2021,Log GDP per capita_2021,Social support_2021,Healthy life expectancy at birth_2021,Freedom to make life choices_2021,Generosity_2021,Perceptions of corruption_2021
Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.724,7.370,0.451,...,NaN,NaN,NaN,2.523,7.695,0.463,52.493,0.382,-0.102,0.924
Albania,4.634,9.142,0.821,65.80,0.529,-0.009,0.875,NaN,NaN,NaN,...,0.754,0.007,0.891,5.117,9.520,0.697,68.999,0.785,-0.030,0.901
Argentina,6.073,10.018,0.862,66.94,0.653,-0.141,0.881,5.961,10.048,0.892,...,0.823,-0.122,0.816,5.929,9.962,0.898,69.000,0.828,-0.182,0.834
Armenia,4.882,9.181,0.760,64.90,0.605,-0.251,0.817,4.652,9.256,0.709,...,NaN,NaN,NaN,5.283,9.487,0.799,67.055,0.825,-0.168,0.629
Australia,7.285,10.703,0.965,71.72,0.891,0.347,0.513,7.254,10.719,0.947,...,0.905,0.210,0.491,7.183,10.796,0.940,73.900,0.914,0.159,0.442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.258,9.701,0.922,...,0.612,NaN,0.811,4.892,9.073,0.861,66.700,0.615,-0.169,0.827
Vietnam,5.422,8.394,0.856,66.02,0.918,0.089,0.754,5.480,8.440,0.805,...,NaN,NaN,NaN,5.411,8.973,0.850,68.034,0.940,-0.098,0.796
Yemen,4.477,8.214,0.825,53.40,0.673,0.011,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.658,7.578,0.832,57.122,0.602,-0.147,0.800
Zambia,3.998,7.871,0.688,45.72,0.682,-0.067,0.948,4.730,7.918,0.624,...,0.750,0.056,0.810,4.073,8.145,0.708,55.809,0.782,0.061,0.823


In [5]:
temp_df = all_data.drop(columns = ['Regional indicator'])
years_present_df = temp_df[(temp_df['year'] >= 2007) & (temp_df['year'] <= 2021)].groupby('Country name').count()[['year']]
countries_to_include = list(years_present_df[years_present_df['year'] > 10].index)
included_years = [i for i in range(2007,2022)]
columns = temp_df.columns[2:]
new_col_names = []
for col in columns:
    for i in included_years:
        year = str(i)
        col_name = f'{col}_{year}'
        new_col_names.append(col_name)

groups = temp_df.groupby('Country name')

num_cols = len(columns)
missing_year_data = [np.nan for i in range(num_cols)]
data = {}
for key, group in groups:
    if key in countries_to_include:
        row_data = []
        group_years = list(group['year'])
        c_group = group.drop(columns = 'Country name')
        c_group = c_group.set_index('year')
        for col in columns:
            col_years = []
            col_vals = []
            for year in included_years:
                try:
                    t = c_group.loc[year,col]
                    if t == t:
                        col_years.append(year)
                        col_vals.append(t)
                except:
                    t = 0  
            if len(col_years) > 0:
                X = pd.DataFrame({'year':col_years})
                Y = pd.DataFrame({col:col_vals})[col]
                col_predictor = LinearRegression().fit(X,Y)
                for year in included_years:
                    try:
                        t = c_group.loc[year,col]
                        if t == t:
                            row_data.append(t)
                        else:
                            predicted_value = col_predictor.predict(pd.DataFrame({'year':[year]}))[0]    
                            row_data.append(predicted_value) 
                    except:
                        predicted_value = col_predictor.predict(pd.DataFrame({'year':[year]}))[0]    
                        row_data.append(predicted_value) 
            else:
                for year in included_years:
                    row_data.append(np.nan) 
        data[key] = row_data
    

reformatted_data = pd.DataFrame.from_dict(data, orient='index',columns = new_col_names)

ttl_nulls = pd.DataFrame(reformatted_data.isnull().sum()).sum()
ttl_vals = reformatted_data.shape[0] * reformatted_data.shape[1]
pct_missing = ttl_nulls / ttl_vals
print(pct_missing)

reformatted_data

0    0.579879
dtype: float64


,Life Ladder_2007,Life Ladder_2008,Life Ladder_2009,Life Ladder_2010,Life Ladder_2011,Life Ladder_2012,Life Ladder_2013,Life Ladder_2014,Life Ladder_2015,Life Ladder_2016,...,dis_gall_2012,dis_gall_2013,dis_gall_2014,dis_gall_2015,dis_gall_2016,dis_gall_2017,dis_gall_2018,dis_gall_2019,dis_gall_2020,dis_gall_2021
Afghanistan,4.558258,3.724000,4.402000,4.758000,3.832,3.783,3.572,3.131,3.983,4.220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,4.634000,5.125414,5.485000,5.269000,5.867,5.510,4.551,4.814,4.607,4.511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Argentina,6.073000,5.961000,6.424000,6.441000,6.776,6.468,6.582,6.671,6.697,6.427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Armenia,4.882000,4.652000,4.178000,4.368000,4.260,4.320,4.277,4.453,4.348,4.325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Australia,7.285000,7.254000,7.336073,7.450000,7.406,7.196,7.364,7.289,7.309,7.250,...,11.02157,10.984307,10.984307,10.984307,11.579724,11.579724,11.579724,12.099242,11.986867,12.119307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,7.423220,6.258000,7.189000,7.478000,6.580,7.067,6.553,6.136,5.569,4.041,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vietnam,5.422000,5.480000,5.304000,5.296000,5.767,5.535,5.023,5.085,5.076,5.062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,4.477000,4.387173,4.809000,4.350000,3.746,4.061,4.218,3.968,2.983,3.826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zambia,3.998000,4.730000,5.260000,4.748332,4.999,5.013,5.244,4.346,4.843,4.348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
wealth_countries = list(wealth_df.sort_values(by='country')['country'].unique())
not_in_wealth = ['Cambodia','Kenya','Morocco','Singapore','Guatemala','Honduras']
def check_gov(x):
    if x in wealth_countries and x not in not_in_wealth:

        
        return True
    else:
        return False
wealth_data = reformatted_data[list(map(check_gov,list(reformatted_data.index)))]
ttl_nulls = pd.DataFrame(wealth_data.isnull().sum()).sum()
ttl_vals = wealth_data.shape[0] * wealth_data.shape[1]
pct_missing = ttl_nulls / ttl_vals
print(pct_missing)
wealth_data

0    0.0
dtype: float64


,Life Ladder_2007,Life Ladder_2008,Life Ladder_2009,Life Ladder_2010,Life Ladder_2011,Life Ladder_2012,Life Ladder_2013,Life Ladder_2014,Life Ladder_2015,Life Ladder_2016,...,povertyRate_2012,povertyRate_2013,povertyRate_2014,povertyRate_2015,povertyRate_2016,povertyRate_2017,povertyRate_2018,povertyRate_2019,povertyRate_2020,povertyRate_2021
Albania,4.634000,5.125414,5.485000,5.269000,5.867,5.510,4.551,4.814,4.607,4.511,...,7.7,7.7,7.7,7.7,7.7,7.7,7.7,7.7,7.7,7.7
Argentina,6.073000,5.961000,6.424000,6.441000,6.776,6.468,6.582,6.671,6.697,6.427,...,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5
Armenia,4.882000,4.652000,4.178000,4.368000,4.260,4.320,4.277,4.453,4.348,4.325,...,13.5,13.5,13.5,13.5,13.5,13.5,13.5,13.5,13.5,13.5
Australia,7.285000,7.254000,7.336073,7.450000,7.406,7.196,7.364,7.289,7.309,7.250,...,12.8,12.8,12.8,12.8,12.8,12.8,12.8,12.8,12.8,12.8
Austria,7.305756,7.181000,7.292555,7.303000,7.471,7.401,7.499,6.950,7.076,7.048,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
United States,7.513000,7.280000,7.158000,7.164000,7.115,7.026,7.249,7.151,6.864,6.804,...,16.8,16.8,16.8,16.8,16.8,16.8,16.8,16.8,16.8,16.8
Uruguay,5.694000,5.664000,6.296000,6.062000,6.554,6.450,6.444,6.561,6.628,6.171,...,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4
Vietnam,5.422000,5.480000,5.304000,5.296000,5.767,5.535,5.023,5.085,5.076,5.062,...,11.6,11.6,11.6,11.6,11.6,11.6,11.6,11.6,11.6,11.6
Zambia,3.998000,4.730000,5.260000,4.748332,4.999,5.013,5.244,4.346,4.843,4.348,...,74.3,74.3,74.3,74.3,74.3,74.3,74.3,74.3,74.3,74.3


In [67]:
wealth_data['wealthGini_2012'].sort_values()

Dominican Republic     0.0
Albania               44.8
Hungary               45.3
Slovakia              46.3
Moldova               47.9
                      ... 
Ukraine               90.1
Egypt                 91.7
Kazakhstan            92.6
Guatemala              NaN
Honduras               NaN
Name: wealthGini_2012, Length: 93, dtype: float64

In [66]:
wealth_data.isnull().sum().sort_values()

Life Ladder_2007                  0
Perceptions of corruption_2013    0
Perceptions of corruption_2014    0
Perceptions of corruption_2015    0
Perceptions of corruption_2016    0
                                 ..
wealthGini_2017                   2
wealthGini_2013                   2
wealthGini_2014                   2
wealthGini_2008                   2
wealthGini_2012                   2
Length: 165, dtype: int64

In [7]:
gov_countries = list(CPDS_df.sort_values(by='country')['country'].unique())

def check_gov(x):
    if x in gov_countries:
        return True
    else:
        return False
gov_data = reformatted_data[list(map(check_gov,list(reformatted_data.index)))]
ttl_nulls = pd.DataFrame(gov_data.isnull().sum()).sum()
ttl_vals = gov_data.shape[0] * gov_data.shape[1]
pct_missing = ttl_nulls / ttl_vals
print(f'Percentage missing = {pct_missing}')

Percentage missing = 0    0.0
dtype: float64


In [34]:
years_included_df = temp_df[(temp_df['year'] >= 2007) & (temp_df['year'] <= 2021)]
print(years_included_df.shape)
pd.DataFrame(years_included_df.isnull().sum()).sort_values(by  = 0, ascending = False).head(100)


(1982, 39)


,0
gov_gap,1580
gov_new,1580
gov_cent2,1578
gov_left1,1578
gov_type,1578
gov_sup,1578
gov_left3,1578
gov_cent3,1578
gov_right3,1578
gov_left2,1578


In [66]:
pd.DataFrame({'year':[1,2,3]})
pd.DataFrame({'vals':[1,2,3]})['vals']

0    1
1    2
2    3
Name: vals, dtype: int64

In [27]:
pd.DataFrame(reformatted_data.isnull().sum()).sort_values(by  = 0, ascending = False).head(100)

,0
gov_left2_2014,93
gov_cent3_2021,93
gov_right3_2008,93
gov_right3_2007,93
gov_left2_2021,93
...,...
gov_right2_2012,93
gov_right2_2011,93
gov_right2_2010,93
gov_right2_2009,93


In [69]:
gov_data.to_csv('final_data/gov_data_with_no_missing_data.csv')

In [26]:
pd.DataFrame(reformatted_data.isnull().sum()).sort_values(by  = 0, ascending = False).head(100)

,0
gov_left2_2014,93
gov_cent3_2021,93
gov_right3_2008,93
gov_right3_2007,93
gov_left2_2021,93
...,...
gov_right2_2012,93
gov_right2_2011,93
gov_right2_2010,93
gov_right2_2009,93


In [56]:
ttl_nulls = pd.DataFrame(reformatted_data.isnull().sum()).sum()
ttl_vals = reformatted_data.shape[0] * reformatted_data.shape[1]
pct_missing = ttl_nulls / ttl_vals
pct_missing

0    0.095688
dtype: float64

In [43]:
def convert_to_int(x):
    return int(x)

CPDS_df['year'] = CPDS_df[['year']].applymap(convert_to_int)
CPDS_df.head()

,year,country,countryn,iso,iso3n,cpds1,poco,eu,emu,gov_right1,...,emprot_temp,prefisc_gini,pretran_gini,postfisc_gini,pop,pop15_64,pop65,elderly,year_13,country_13
0,1960,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10249.099609,6353.600098,849.000000,8.283654,1960.0,Australia
1,1961,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10452.099609,6486.899902,868.099976,8.305508,1961.0,Australia
2,1962,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10649.500000,6631.100098,886.500000,8.324334,1962.0,Australia
3,1963,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10851.200195,6771.700195,905.400024,8.343778,1963.0,Australia
4,1964,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,11071.299805,6919.500000,920.000000,8.309773,1964.0,Australia


In [19]:
X = pd.DataFrame({'year':[1]})
test_x = pd.DataFrame({'year':[10]})
Y = pd.DataFrame({'temp':[]})['temp']
model = LinearRegression().fit(X,Y)

model.predict(test_x)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [18]:
len(Y)

1